In [1]:
#これはCUIです

import pandas as pd
import os
import csv
import matplotlib.pyplot as plt
from PIL import Image
import matplotlib.cm as cm
import seaborn as sns
import tkinter as tk
from tkinterdnd2 import *

#参照データのインポート
sheets = ["Action List", "Action Type", "Action Result", "Qualifier3", "Qualifier4", "Qualifier5"]
ref = pd.read_excel(os.getcwd() + "/Resources/Opta Data ID.xlsx", sheet_name = sheets)
alist = pd.DataFrame(ref["Action List"])
atype = pd.DataFrame(ref["Action Type"])
aresult = pd.DataFrame(ref["Action Result"])
q3 = pd.DataFrame(ref["Qualifier3"])
q4 = pd.DataFrame(ref["Qualifier4"])
q5 = pd.DataFrame(ref["Qualifier5"])
#参照データ定義

#PLID,TEAMIDのインポート
psheets = ["PLID", "TID", "Venue"]
ptid = pd.read_excel(os.getcwd() +"/Resources/Japan Rugby League One Player ID.xlsx", sheet_name = psheets)
tid = pd.DataFrame(ptid["TID"])
venue = pd.DataFrame(ptid["Venue"])
plid = pd.DataFrame(ptid["PLID"])

def rhimport(filename):
    #ローデータのインポート
    Opta = str(filename)
    #Opta = "/Users/isakakou/Desktop/Advance Superscouts/2022_LeagueOne_Div3/97694_KYUDvSHBS_advanced_superscout.xlsx"
    df = pd.DataFrame(pd.read_excel(Opta, header=1))
   
    #ヘッダー、IDの変更
    cl_name = ['FXID', 'ATFTSC','ATFTSC_agg', 'awayteam','FxATID', 'FxATID_agg','FxDate', 'FxHTID','FxHTID_agg', 'FxTID','FxTID_agg', 'FxWeek','FxWeek_agg', 'hometeam','HTFTSC', 'HTFTSC_agg','Leadtime', 'Leadtime_agg','Pathname', 'REFID', 'REFID_agg', 'REFNAME','ActionRow_action','Actionresult','ActionType','ActionRow_advantage','ActionRow_assoc_player','ActionRow_flag', 'ActionRow_FXID','ActionRow_ID', 'MatchTime','Metres','period','player_advantage','PlayNum', 'ActionRow_PLID','ps_endstamp','ps_timestamp','psID','qualifier3','qualifier4','qualifier5','score_advantage','sequence_id','SetNum','ActionRow_team_id','x_coord','x_coord_end','y_coord','y_coord_end', 'Player_Club','Player/@MINS', 'Player_PLFORN','Player_PLID', 'Player_PLSURN','Player_PosID', 'Player_ShirtNo','Player_TEAMNAME']
    df.columns = cl_name

    df = df.drop(['REFID_agg','FxWeek_agg','HTFTSC', 'HTFTSC_agg',
       'Leadtime', 'Leadtime_agg', 'Pathname','FxHTID', 'FxHTID_agg', 'FxTID', 'FxTID_agg', 'ATFTSC', 'ATFTSC_agg', 'FxATID', 'FxATID_agg','Player_PLFORN', 'Player_PLSURN', 'Player_PosID',
       'Player_ShirtNo', 'Player_TEAMNAME'], axis=1)
    df.qualifier3 = df.qualifier3.map(q3.set_index('ID').Definition)
    df.qualifier4 = df.qualifier4.map(q4.set_index('ID').Definition)
    df.qualifier5 = df.qualifier5.map(q5.set_index('ID').Definition)
    df.Actionresult = df.Actionresult.map(aresult.set_index('ID').Definition)
    df.ActionType = df.ActionType.map(atype.set_index('ID').Definition)
    df.ActionRow_action = df.ActionRow_action.map(alist.set_index('ID').Definition)
    df.ActionRow_team_id = df.ActionRow_team_id.map(tid.set_index('team_id').team_name)
    df.ActionRow_PLID = df.ActionRow_PLID.map(plid.set_index('players_id').player_known_name)
    return(df)

        #表を作成
def make_table(filename,action,teamname):
    df = rhimport(filename)
    df = df.fillna('NA')
    if teamname == df.hometeam[10]:
        opp = df.awayteam[10]
    else:
        opp = df.hometeam[10]
    #df = pd.get_dummies(df)
    tabledf = df.loc[(df['ActionRow_team_id'] == teamname)]
    
    tabledf = tabledf.drop(['ps_endstamp', 'ps_timestamp', 'psID','score_advantage', 'sequence_id', 'SetNum','ActionRow_advantage','ActionRow_assoc_player','ActionRow_flag','ActionRow_FXID','ActionRow_ID','FxDate','FxWeek','REFID','Player_Club','Player/@MINS','Player_PLID'], axis=1)
    dfaction = tabledf.loc[tabledf['ActionRow_action'] == action]
    a = pd.crosstab([dfaction['qualifier3'],dfaction['Actionresult']],dfaction['ActionType'],margins=True)
    b = pd.crosstab([dfaction['qualifier3'],dfaction['Actionresult']],dfaction['ActionType'])
    if action == 'Lineout Throw':
        a = pd.crosstab([dfaction['Metres'],dfaction['Actionresult']],dfaction['ActionType'],margins=True)
        a = a.reindex(columns=['Throw Front', 'Throw Middle', 'Throw Back', 'Throw 15m+'])
        b = pd.crosstab(dfaction['Metres'],dfaction['ActionType'])
        b = b.reindex(columns=['Throw Front', 'Throw Middle', 'Throw Back', 'Throw 15m+'])
    print(a)
    if action == 'Ruck':
        a = pd.crosstab(dfaction['qualifier3'],dfaction['Actionresult'],margins=True)
        
    if action == 'Penalty':
        pentable = dfaction.drop(['Actionresult', 'qualifier3','qualifier4','ActionRow_team_id'], axis=1)
        pentable.to_excel(os.getcwd()+"/Files/" +action +' of '+teamname + '.xlsx', index=False)
        
        
    a.to_csv(os.getcwd()+"/Files/" +action +' of '+teamname +' v ' + opp + '.csv')
    plt.gcf().subplots_adjust(wspace=4)
    plt.savefig(os.getcwd()+"/Files/" +action +'s of '+teamname +' v ' + opp +'_table.png')
    sns.heatmap(b,cmap='Blues', annot=True,annot_kws={'size':20})
    plt.clf()
        
    #a.plot.bar(stacked=True)

    #プロット用関数定義-----
def plotter(filename, action, teamname):
    df = rhimport(filename)
    if teamname == df.hometeam[10]:
        opp = df.awayteam[10]
    else:
        opp = df.hometeam[10]
    cmap = plt.get_cmap("magma")
    fig = plt.figure(figsize=(7, 10))
    ax = fig.add_subplot(1, 1,1)
    plts = df.loc[(df['ActionRow_action'] == action)]
    plts = plts.loc[(plts['ActionRow_team_id'] == teamname)]
    plt.xlim(0, 68)
    plt.ylim(0,100)
    cl = ['y','m', 'c', 'r', 'g', 'b', (0,0.3,0.5), (0,0,0)]
    #始点終点の場合

    #Kickが入力された場合の処理-----
    if action == "Kick":
        kicktypes = ['Bomb', 'Chip', 'Cross Pitch', 'Territorial', 'Low', 'Box', 'Touch Kick','Kick Error']
        kicks =  plts.loc[(plts['qualifier3'] =='Kick in Play') | (plts['qualifier3'] == 'Kick in Play (Own 22)')]
        #kicks.y_coord_end.mask(kicks.y_coord_end>=68,68)
        print(kicks)
        for i in range(len(kicktypes)):
            plts = kicks.loc[kicks['ActionType']== kicktypes[i]]
            plt.quiver(plts['y_coord'], plts["x_coord"], (plts['y_coord_end']-plts['y_coord']), (plts["x_coord_end"]-plts['x_coord']), angles="xy", scale_units='xy',label= kicktypes[i], color = cl[i], scale=1,width=0.008, headwidth=5, headlength = 8, headaxislength = 7, pivot='tail')
    #Kickの処理ここまで------

    elif action == "Ruck":
        rspeed = ['0-1 Seconds','1-2 Seconds','3-4 Seconds','4-5 Seconds','5-6 Seconds','6+ Seconds', "N/A Ruck Speed"]
        for i in range(len(rspeed)):
            r = plts.loc[plts['qualifier4'] == rspeed[i]]
            plt.scatter(r['y_coord'], r["x_coord"], label=rspeed[i] ,marker = 'D', color=cl[i], s=150)
            
    elif action == 'Carry':
        playerlist = plts['ActionRow_PLID']
        player = playerlist.drop_duplicates(keep='last')
        player = player.reset_index(drop=True)
        print(player)
        for pl in range(len(player)):
            fig.patch.set_facecolor('white')
            im = Image.open(os.getcwd()+"/Resources/FIELD image.JPEG")
            xlim = ax.get_xlim()
            ylim = ax.get_ylim()
            plt.imshow(im, extent=[*xlim, *ylim], aspect='auto', alpha=0.6)
            p = plts.loc[plts['ActionRow_PLID']== player[pl]]
            at = p.loc[p['ActionType']]
            at = at.reset_index(drop=True)
            print(player)
            for i in range(len(p)):
                plt.quiver(p['y_coord'], p["x_coord"], (p['y_coord_end']-p['y_coord']), (p["x_coord_end"]-p['x_coord']), angles="xy", scale_units='xy', label = at, color = red, scale=1,width=0.008, headwidth=5, headlength = 8, headaxislength = 7, pivot='tail')
                plt.title(p[i])
            plt.legend()
            plt.show()
            fig = plt.figure(figsize=(7, 10))
            ax = fig.add_subplot(1, 1,1)
            plt.xlim(0, 68)
            plt.ylim(0,100)
            

    else:
        plt.scatter(plts['y_coord'],plts['x_coord'], label=action,color = cmap(190), marker = 'D', s = 230)

    #共通部分
    fig.patch.set_facecolor('white')
    im = Image.open(os.getcwd()+"/Resources/FIELD image.JPEG")
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    plt.imshow(im, extent=[*xlim, *ylim], aspect='auto', alpha=0.6)
    # グラフタイトル
    plt.title(action +'s of '+teamname +' v ' + opp, fontsize=18)
    # Y軸ラベル
    plt.ylabel("Attacking This Way-------->", fontsize=19)
    lgd = plt.legend(loc='upper left', bbox_to_anchor=(1, 1), fontsize = 12)
    plt.gcf().subplots_adjust(wspace=4)
    plt.savefig(os.getcwd()+"/Files/"+ action +'s of '+teamname +' v ' + opp + '_plot.png',bbox_inches='tight')
    plt.show()
    plt.clf()
        # 散布図表示
    #プロット用関数定義終了-----


def linear(filename,action, teamname):    
    df = rhimport(filename)
    if teamname == df.hometeam[10]:
        opp = df.awayteam[10]
    else:
        opp = df.hometeam[10]
    fig = plt.figure(figsize=(12, 15))
    ax = fig.add_subplot(1, 1,1)
    plts = df.loc[(df['ActionRow_action'] == action)]
    plts = plts.loc[(plts['ActionRow_team_id'] == teamname)]
    plt.ylim(0,100)
    plt.xlim(0,70)
    cl = ['y','m', 'c', 'r', 'g', 'b', (0,0.3,0.5), (0,0,0)]
    fpoint = plts.drop_duplicates(subset=['SetNum'],keep='first')
    plt.scatter(fpoint['y_coord'], fpoint['x_coord'], marker = '*' ,c = 'blue',s =180, zorder=2)
    cmap = plt.get_cmap("magma")
    l=1

    #始点終点の場合
    if action == 'Ruck':
        for sn in plts.SetNum:
            sets = plts.loc[plts['SetNum'] == sn]
            #rspeed = ['0-1 Seconds','1-2 Seconds','3-4 Seconds','4-5 Seconds','5-6 Seconds','6+ Seconds', "N/A Ruck Speed"]
            x = sets['y_coord']
            y = sets["x_coord"]
            plt.plot(x, y,lw = 3,marker = 'o',color = cmap(l), markersize = 12,zorder=1)
            l = l+2
            
    fig.patch.set_facecolor('white')
    im = Image.open(os.getcwd()+"/Resources/FIELD image.JPEG")
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    plt.imshow(im, extent=[*xlim, *ylim], aspect='auto', alpha=0.6)
    # グラフタイトル
    plt.title(action +'s of '+teamname +' v ' + opp, fontsize=18)
    # Y軸ラベル
    plt.ylabel("Attacking This Way-------->", fontsize=19)
    plt.colorbar()
    # 散布図表示
    plt.gcf().subplots_adjust(wspace=4)

    plt.savefig(os.getcwd()+"/Files/"+ action +'s of '+teamname +' v ' + opp + '_line.png',bbox_inches='tight')
    plt.show()
    plt.clf()


"""----------------------------ここからはグラフィック-----------------------------------"""

def name_in(event):
    boxtext.set(event.data)
    
def imported():
    def check_v():
        value = chk_bln.get()
        if value == True:
            action.append(value)
        print(value)
        
    def tsel(variable):
        print(variable)
        tn.set(variable)
        
    def analyze(filename, action, teamname):
        plotter(filename, action, teamname)
        plt.clf()
        linear(filename, action, teamname)
        plt.clf()
        make_table(filename, action,teamname)
        plt.clf()
        
    base.destroy()
    filename = boxtext.get()
    filename = filename.rstrip('}').lstrip('{')
    txt.set(filename)
    df = rhimport(filename)
    TeamA = str(df.hometeam[10])
    #TeamA = ("Toyota Industries Shuttles Aichi")
    TeamB = str(df.awayteam[10])
    REF_NAME = str(df.REFNAME[10])
    t_list =[]
    t_list = [TeamA, TeamB]
    A_menu = tk.Tk()
    A_menu.geometry('700x700')
    A_menu.title('Analysis')
    variable = tk.StringVar(A_menu)
    variable.set(t_list)
    tn = tk.StringVar(A_menu)
    sel_team = tk.OptionMenu(A_menu, variable, *t_list, command=tsel)
    sel_team.config(height=4,width=100, font=('Helvetica', 18))
    sel_team.pack()
    
    Possession =tk.Button(A_menu, text="Possession", command=lambda:analyze(filename, "Possession", str(tn.get()))).pack()
    Sequence = tk.Button(A_menu, text="Sequence", command=lambda:analyze(filename, "Sequence", str(tn.get()))).pack()
    Scrum=tk.Button(A_menu, text="Scrum", command=lambda:analyze(filename, "Scrum", str(tn.get()))).pack()
    Ruck=tk.Button(A_menu, text="Ruck", command=lambda:analyze(filename, "Ruck", str(tn.get()))).pack()
    Maul=tk.Button(A_menu, text="Maul", command=lambda:analyze(filename, "Maul", str(tn.get()))).pack()
    CounterAttack=tk.Button(A_menu, text="Counter Attack", command=lambda:analyze(filename, "Counter Attack", str(tn.get()))).pack()
    DefensiveExit=tk.Button(A_menu, text="Defensive Exit", command=lambda:analyze(filename, "Defensive Exit", str(tn.get()))).pack()
    Attacking22Entry=tk.Button(A_menu, text="Attacking 22 Entry", command=lambda:analyze(filename, "Attacking 22 Entry", str(tn.get()))).pack()
    RestartKick=tk.Button(A_menu, text="Restart Kick", command=lambda:analyze(filename, "Restart Kick", str(tn.get()))).pack()
    LineoutThrow=tk.Button(A_menu, text="Lineout Throw", command=lambda:analyze(filename, "Lineout Throw", str(tn.get()))).pack()
    LineoutCatch=tk.Button(A_menu, text="Lineout Catch", command=lambda:analyze(filename, "Lineout Catch", str(tn.get()))).pack()
    Penalty=tk.Button(A_menu, text="Penalty", command=lambda:analyze(filename, "Penalty", str(tn.get()))).pack()
    Card=tk.Button(A_menu, text="Card", command=lambda:analyze(filename, "Card", str(tn.get()))).pack()
    Carry=tk.Button(A_menu, text="Carry", command=lambda:analyze(filename, "Carry", str(tn.get()))).pack()
    Pass=tk.Button(A_menu, text="Pass", command=lambda:analyze(filename, "Pass", str(tn.get()))).pack()
    Collection=tk.Button(A_menu, text="Collection", command=lambda:analyze(filename, "Collection", str(tn.get()))).pack()
    Kick=tk.Button(A_menu, text="Kick", command=lambda:analyze(filename, "Kick", str(tn.get()))).pack()
    GoalKick=tk.Button(A_menu, text="Goal Kick", command=lambda:analyze(filename, "Goal Kick", str(tn.get()))).pack()
    Tackle=tk.Button(A_menu, text="Tackle", command=lambda:analyze(filename, "Tackle", str(tn.get()))).pack()
    MissedTackle=tk.Button(A_menu, text="Missed Tackle", command=lambda:analyze(filename, "Missed Tackle", str(tn.get()))).pack()
    Turnovers=tk.Button(A_menu, text="Turnovers", command=lambda:analyze(filename, "Turnovers", str(tn.get()))).pack
    RefereeReview=tk.Button(A_menu, text="Referee Review", command=lambda:analyze(filename, "Referee Review", str(tn.get()))).pack()
    DefensiveAction=tk.Button(A_menu, text="Defensive Action", command=lambda:analyze(filename, "Defensive Action", str(tn.get()))).pack()
    PlayerLeavesField=tk.Button(A_menu, text="Player Leaves Field", command=lambda:analyze(filename, "Player Leaves Field", str(tn.get()))).pack()
    PlayerEntersField=tk.Button(A_menu, text="Player Enters Field", command=lambda:analyze(filename, "Player Enters Field", str(tn.get()))).pack()
    RuckOOA=tk.Button(A_menu, text="Ruck OOA", command=lambda:analyze(filename, "Ruck OOA", str(tn.get()))).pack()
    
    A_menu.mainloop()
    
"""ウィンドウの設定"""
base = TkinterDnD.Tk()
base.geometry('400x200')
base.title('Super Scout Advance Reader')
description = tk.Label(text= 'Drag & Drop File').pack()

"""StringVarの設定"""
txt = tk.StringVar(base)
txt.set("PATH will be shown Here")
boxtext =tk.StringVar(base)

"""Editboxの設定"""
editbox = tk.Entry(text=boxtext)
editbox.drop_target_register(DND_FILES)
editbox.dnd_bind('<<Drop>>',name_in)


"""メインウィンドウ"""
ibutton = tk.Button(base, text='IMPORT',command = imported)
fname = tk.Label(textvariable=txt, width=80)
editbox.pack()
ibutton.pack()
fname.pack()

base.mainloop()

In [4]:
import xml.etree.ElementTree as ET 
import pandas as pd
import os
import csv

alist = pd.read_csv(os.getcwd() + "/Resources/ActionList.csv",dtype=str)
atype = pd.read_csv(os.getcwd() + "/Resources/ActionType.csv",dtype=str)
aresult = pd.read_csv(os.getcwd() + "/Resources/ActionResult.csv",dtype=str)
q3 = pd.read_csv(os.getcwd() + "/Resources/Qualifier3.csv",dtype=str)
q4 = pd.read_csv(os.getcwd() + "/Resources/Qualifier4.csv",dtype=str)
q5 = pd.read_csv(os.getcwd() + '/Resources/Qualifier5.csv',dtype=str)


# XMLファイルを解析
tree = ET.parse('/Users/isakakou/Desktop/932604_OSPRvLEIC_advanced_superscout.xml') 

# XMLを取得
root = tree.getroot()
df = pd.DataFrame(index=[],columns = ['ID','FXID','PLID','team_id','ps_timestamp','ps_endstamp','MatchTime','psID','period',
           'x_coord','y_coord','x_coord_end','y_coord_end','action','ActionType','Actionresult',
           'qualifier3','qualifier4','qualifier5','Metres','PlayNum','SetNum',
           'sequence_id','player_advantage','score_advantage','flag','advantage','assoc_player'])

for action in root.iter('ActionRow'):
    s = pd.Series([action.get('ID'),action.get('FXID'),action.get('PLID'),action.get('team_id'),
                   action.get('ps_timestamp'),action.get('ps_endstamp'),action.get('MatchTime'),action.get('psID'),
                   action.get('period'),action.get('x_coord'),action.get('y_coord'),action.get('x_coord_end'),action.get('y_coord_end'),
                   action.get('action'),action.get('ActionType'),action.get('Actionresult'),action.get('qualifier3'),
                   action.get('qualifier4'),action.get('qualifier5'),action.get('Metres'),action.get('PlayNum'),action.get('SetNum'),
                   action.get('sequence_id'),action.get('player_advantage'),action.get('score_advantage'),action.get('flag'),
                   action.get('advantage'),action.get('assoc_player')], index = df.columns)
    df = df.append(s, ignore_index=True)
df.qualifier3 = df.qualifier3.map(q3.set_index('ID').Definition)
df.qualifier4 = df.qualifier4.map(q4.set_index('ID').Definition)
df.qualifier5 = df.qualifier5.map(q5.set_index('ID').Definition)
df.Actionresult = df.Actionresult.map(aresult.set_index('ID').Definition)
df.ActionType = df.ActionType.map(atype.set_index('ID').Definition)
df.action = df.action.map(alist.set_index('ID').Definition)
for da in df.action:
    print(da)
# df.to_csv('/Users/isakakou/Desktop/optaTest.csv', header=True, index=False)

Clock Stoppage
Restart Kick
Collection
Tackle
Tackle
Carry
Ruck
Ruck OOA
Ruck OOA
Ruck OOA
Ruck OOA
Collection
Kick
Playmaker Options
Defensive Exit
Possession
Collection
Pass
Collection
Tackle
Carry
Defensive Action
Ruck
Ruck OOA
Ruck OOA
Ruck OOA
Ruck OOA
Ruck OOA
Collection
Kick
Playmaker Options
Penalty
Sequence
Possession
Kick
Turnovers
Sequence
Scrum
Offensive Scrum
Offensive Scrum
Offensive Scrum
Offensive Scrum
Offensive Scrum
Offensive Scrum
Offensive Scrum
Offensive Scrum
Defensive Scrum
Defensive Scrum
Defensive Scrum
Defensive Scrum
Defensive Scrum
Defensive Scrum
Defensive Scrum
Defensive Scrum
Sequence
Scrum
Offensive Scrum
Offensive Scrum
Offensive Scrum
Offensive Scrum
Offensive Scrum
Offensive Scrum
Offensive Scrum
Offensive Scrum
Defensive Scrum
Defensive Scrum
Defensive Scrum
Defensive Scrum
Defensive Scrum
Defensive Scrum
Defensive Scrum
Defensive Scrum
Penalty
Sequence
Kick
Sequence
Attacking 22 Entry
Lineout Throw
Lineout Catch
Maul
Collection
Pass
Playmaker Optio